In [22]:
from spamdfba import toolkit as tk
from spamdfba import toymodels as tm
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import pickle
import os
import warnings
import rich
import multiprocessing as mp
import json

In [5]:
NUM_CORES = 8
warnings.filterwarnings("ignore")


## Toy-Exoenzyme-Single-Agent

This toy community is designed to emulate a case that microbial cells are grown on a mixture of starch and glucose in a chemostat with a very low dilution rate of 0.0001, practically a batch system, Figure 3A-C, The strains are capable of secreting amylase to degrade the available starch. However, producing amylase is an energy-consuming step in the organism’s metabolism and it requires ATP that would otherwise be used in biomass production. 

First, we need to define the agents. The agents need a metabolic model (cobra model) which is defined in Toy_Model.py. The agents
also need a name, a neural network class, not instance as a pytorch module, clip which shows the threshold for clipping the gradients, and a learning rate. you also need to define what environment states you want your agent to sense and also what is the actions that the agents can take. Look below for an example of defining an agent.
additionally, you can look at toolkit.py for more information on defining agents.

As the first case, we consider only one agent in the system:

![Single-agent](./single_agent.png)




In [7]:
agent1=tk.Agent("agent1",
                model=tm.ToyModel_SA.copy(),
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=4,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1' ,'Glc', 'Starch'],
                actions=["Amylase_e"],
                gamma=1,
                tau=0.1
                )

agents=[agent1]


After defining the agents, we need to define the environment. The environment needs a list of agents, inictial conditions and a dictionary representing extracellular reactions as well as the duration of an episode and the time resoloution of the DFBA algorithm. More information on defining the environment can be found in toolkit.py. Look below for an example of defining the environment.

In [8]:
env_1=tk.Environment(name="Toy-Exoenzyme-Single-agents",
                    agents=agents,
                    dilution_rate=0.0001,
                    initial_condition={"Glc":100,"agent1":0.1,"Starch":10},
                    inlet_conditions={"Starch":10},
                    extracellular_reactions=[{"reaction":{
                    "Glc":10,
                    "Starch":-0.1,},
                    "kinetics": (tk.general_kinetic,("Starch","Amylase"))}],
                     dt=0.1,
                     number_of_batches=5000,
                     episodes_per_batch=int(NUM_CORES/2),)

The following species are not in the community: ['Starch']


Now we can start the training loop. The training loop needs the environment, the number of episodes, the number of steps per episode, and the number of steps for each gradient update. The training loop will return the rewards for each episode and the average reward for each episode. Look below for an example of training loop.

In [11]:
sim_1=tk.Simulation(name=env_1.name,
                  env=env_1,
                  save_dir="./Results/",
                  )

In [13]:
sim_1.run()

Hold on, bringing the creitc network to range ...
Done!
Batch 0 finished:
agent1 return was:  -664.5209123803668
Batch 1 finished:
agent1 return was:  -663.5599404211715
Batch 2 finished:
agent1 return was:  -669.4157698012241
Batch 3 finished:
agent1 return was:  -656.4026235980409
Batch 4 finished:
agent1 return was:  -659.3187908529303
Batch 5 finished:
agent1 return was:  -656.3232269483684
Batch 6 finished:
agent1 return was:  -659.2128032726396
Batch 7 finished:
agent1 return was:  -661.5820917027219
Batch 8 finished:
agent1 return was:  -651.647612625643
Batch 9 finished:
agent1 return was:  -648.437141315243
Batch 10 finished:
agent1 return was:  -635.772870920295
Batch 11 finished:
agent1 return was:  -637.9996842300541
Batch 12 finished:
agent1 return was:  -636.6951170432906
Batch 13 finished:
agent1 return was:  -640.4567959845842
Batch 14 finished:
agent1 return was:  -643.110630736408
Batch 15 finished:
agent1 return was:  -644.1133659082308
Batch 16 finished:
agent1 retu

In [20]:
fig_single_agent=sim_1.plot_learning_curves()

In [23]:
fig_single_agent.write_image(env_1.name+"_learning_curve.svg")

ValueError: 
Image export using the "kaleido" engine requires the kaleido package,
which can be installed using pip:
    $ pip install -U kaleido


In [24]:
time_single_agent=sim_1.print_training_times()

                        Simulation times                        
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Level        ┃ Mean(s)               ┃ STD(s)                ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━┩
│ Optimization │ 0.0003498440815448762 │ 6.495468274710666e-05 │
│ Step         │ 0.0005561942130148409 │ 9.948172783878535e-05 │
│ Batch        │ 1.192914434671402     │ 0.20728934267366944   │
│ Simulation   │ 6135.182281970978     │ NA                    │
└──────────────┴───────────────────────┴───────────────────────┘

In [25]:
with open(env_1.name+"_Runtime.json","w") as f:
    json.dump(time_single_agent,f)

## Toy-Exoenzyme-Two-Agent
This environment is similar to the previous environment but it has two independent agent that are similar in terms of metabolic capabilities.

![Two-agents](./Two_agents.png)

In [26]:
agent1=tk.Agent("agent1",
                model=tm.ToyModel_SA.copy(),
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=4,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1','agent2' ,'Glc', 'Starch'],
                actions=["Amylase_e"],
                gamma=1,
                tau=0.1
                )

agent2=tk.Agent("agent2",
                model=tm.ToyModel_SA.copy(),
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=4,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1','agent2', 'Glc', 'Starch'],
                actions=["Amylase_e"],
                gamma=1,
                tau=0.1
                )

agents=[agent1,agent2]

env_2=tk.Environment(name="Toy-Exoenzyme-Two-agents",
                    agents=agents,
                    dilution_rate=0.0001,
                    initial_condition={"Glc":100,"agent1":0.1,"agent2":0.1,"Starch":10},
                    inlet_conditions={"Starch":10},
                    extracellular_reactions=[{"reaction":{
                    "Glc":10,
                    "Starch":-0.1,},
                    "kinetics": (tk.general_kinetic,("Starch","Amylase"))}],
                           dt=0.1,
                           number_of_batches=5000,
                           episodes_per_batch=int(NUM_CORES/2),
                           )



The following species are not in the community: ['Starch']


In [27]:
sim_2=tk.Simulation(name=env_2.name,
                  env=env_2,
                  save_dir="./Results/",
                  )


In [28]:
sim_2.run()

Hold on, bringing the creitc network to range ...
Done!
Hold on, bringing the creitc network to range ...
Done!
Batch 0 finished:
agent1 return was:  -676.398503398257
agent2 return was:  -699.1062407617163
Batch 1 finished:
agent1 return was:  -679.972052650075
agent2 return was:  -697.940473495102
Batch 2 finished:
agent1 return was:  -673.7934140726132
agent2 return was:  -703.8702564946761
Batch 3 finished:
agent1 return was:  -671.9064611821393
agent2 return was:  -709.4563511931123
Batch 4 finished:
agent1 return was:  -688.3148352911585
agent2 return was:  -707.1471894809868
Batch 5 finished:
agent1 return was:  -674.9548643893252
agent2 return was:  -719.4544003079012
Batch 6 finished:
agent1 return was:  -677.1086467042621
agent2 return was:  -712.3459030847919
Batch 7 finished:
agent1 return was:  -677.8570266783959
agent2 return was:  -705.1225065431831
Batch 8 finished:
agent1 return was:  -680.3673387312069
agent2 return was:  -714.4289736655686
Batch 9 finished:
agent1 re

In [ ]:
fig_two_agents=sim_2.plot_learning_curves()

In [ ]:
fig_two_agents.write_image(env_2.name+"_learning_curve.svg")

In [29]:
time_two_agent=sim_2.print_training_times()

                        Simulation times                         
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Level        ┃ Mean(s)               ┃ STD(s)                 ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Optimization │ 0.0004752251323282723 │ 6.358572031670175e-05  │
│ Step         │ 0.0014352515552341965 │ 0.00018915827331749974 │
│ Batch        │ 3.0117019214630125    │ 0.4198159922654137     │
│ Simulation   │ 15380.066538095474    │ NA                     │
└──────────────┴───────────────────────┴────────────────────────┘

In [30]:
with open(env_2.name+"_Runtime.json","w") as f:
    json.dump(time_two_agent,f)

## Toy-Exoenzyme-Five-Agents

This example expands the number of agents to 5. Otherwise, it is similar to the previous cases.

![Two-agents](./Five_agents.png)

In [31]:
agent1=tk.Agent("agent1",
                model=tm.ToyModel_SA.copy(),
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=4,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1','agent2','agent3','agent4','agent5','Glc', 'Starch'],
                actions=["Amylase_e"],
                gamma=1,
                tau=0.1
                )

agent2=tk.Agent("agent2",
                model=tm.ToyModel_SA.copy(),
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=4,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1','agent2','agent3','agent4','agent5', 'Glc', 'Starch'],
                actions=["Amylase_e"],
                gamma=1,
                tau=0.1
                )

agent3=tk.Agent("agent3",
                model=tm.ToyModel_SA.copy(),
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=4,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1','agent2','agent3','agent4','agent5', 'Glc', 'Starch'],
                actions=["Amylase_e"],
                gamma=1,
                tau=0.1
                )
agent4=tk.Agent("agent4",
                model=tm.ToyModel_SA.copy(),
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=4,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1','agent2','agent3','agent4','agent5', 'Glc', 'Starch'],
                actions=["Amylase_e"],
                gamma=1,
                tau=0.1
                )
agent5=tk.Agent("agent5",
                model=tm.ToyModel_SA.copy(),
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=4,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1','agent2','agent3','agent4','agent5', 'Glc', 'Starch'],
                actions=["Amylase_e"],
                gamma=1,
                tau=0.1
                )

agents=[agent1,agent2,agent3,agent4,agent5]


env_5=tk.Environment(name="Toy-Exoenzyme-Five-agents",
                  agents=agents,
                  dilution_rate=0.0001,
                  initial_condition={"Glc":100,"agent1":0.1,"agent2":0.1,"agent3":0.1,"agent4":0.1,"agent5":0.1,"Starch":10},
                  inlet_conditions={"Starch":10},
                  extracellular_reactions=[{"reaction":{
                      "Glc":10,
                  "Starch":-0.1,},
                  "kinetics": (tk.general_kinetic,("Starch","Amylase"))}],
                         dt=0.1,
                         number_of_batches=5000,
                         episodes_per_batch=int(NUM_CORES/2),
                         )                 

The following species are not in the community: ['Starch']


In [32]:
sim_5=tk.Simulation(name=env_5.name,
                  env=env_5,
                  save_dir="./Results/",
                  )

In [33]:
sim_5.run()

Hold on, bringing the creitc network to range ...
Done!
Hold on, bringing the creitc network to range ...
Done!
Hold on, bringing the creitc network to range ...
Done!
Hold on, bringing the creitc network to range ...
Done!
Hold on, bringing the creitc network to range ...
Done!
Batch 0 finished:
agent1 return was:  -883.9960557247003
agent2 return was:  -792.2557300214212
agent3 return was:  -779.9249765061945
agent4 return was:  -831.1397972396596
agent5 return was:  -972.7235034537124
Batch 1 finished:
agent1 return was:  -883.496327969325
agent2 return was:  -793.9717884036716
agent3 return was:  -783.1249678574763
agent4 return was:  -831.4335562178511
agent5 return was:  -977.2270205288862
Batch 2 finished:
agent1 return was:  -883.7579947044514
agent2 return was:  -793.0141368087891
agent3 return was:  -781.4031612013408
agent4 return was:  -832.3717727253063
agent5 return was:  -976.7307868936057
Batch 3 finished:
agent1 return was:  -883.9714631844265
agent2 return was:  -793.

In [ ]:
fig_five_agents=sim_5.plot_learning_curves()

NameError: name 'sim' is not defined

In [ ]:
fig_five_agents.write_image(env_5.name+"_learning_curve.svg")

In [ ]:
time_five_agents=sim_5.print_training_times()

                        Simulation times                         
┏━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Level        ┃ Mean(s)               ┃ STD(s)                 ┃
┡━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ Optimization │ 0.0003839415240406987 │ 3.695389236778001e-05  │
│ Step         │ 0.0026809743411540968 │ 0.00025109156605024595 │
│ Batch        │ 5.460161922693253     │ 0.45027560183512977    │
│ Simulation   │ 5603.685482978821     │ NA                     │
└──────────────┴───────────────────────┴────────────────────────┘

In [ ]:
with open(env_5.name+"_Runtime.json","w") as f:
    json.dump(time_five_agents,f)

## Toy-Exoenzyme-Five-Agents-with-mass-transfer

In corporation spacial information in to the current model could be challenging. However, we can simulate a simple case of mass transfer using a trick. Right now, an environment object can include any arbitrary extracellular reactions with an arbitrary kinetics. The way we incorporated mass transfer effect is just to qualitatively see how microbial strategies will change depending on the rate of mass transfer. In this approach, each agent is surrounded by a specific compartment. An agent directly exchange the metabolites of interest with its compartment. The compartments then exchange materials with the shared metabolite pool through a defined exchange reaction with an arbitrary kinetics. In this case the rate of these "reactions" are defined by K*(Ca-Cs), where K is a mass transfer coefficient for the compound in the given environment. Ca represents the concentration of the metabolite in the agent's compartment and Cs represents the concentration of the same compound in the shared compartment: Agent x produces amylase_x and releases it to its compartment. amylase_x degrades starch to glucose_x. glucose_x now can be exchanged with the surroundings through an imaginary reaction glucose_x. In this way, the products that are produced by agent_x will be released to the environment according to a kinetic rather than being instantaneous. In the starch amylase case this could be beneficial to the agent as the broken down glucose won't be available to other agents as fast! The main advantage of this method is that it requires no change in the code! 

In [ ]:

toy_model_1=tm.ToyModel_SA_1.copy()
toy_model_2=tm.ToyModel_SA_2.copy()
toy_model_3=tm.ToyModel_SA_3.copy()
toy_model_4=tm.ToyModel_SA_4.copy()
toy_model_5=tm.ToyModel_SA_5.copy()


agent1=tk.Agent("agent1",
                model=toy_model_1,
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=4,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1','agent2','agent3','agent4','agent5' ,'Glc_1', 'Starch'],
                actions=["Amylase_1_e"],
                gamma=1,
                tau=0.1
                )

agent2=tk.Agent("agent2",
                model=toy_model_2,
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=4,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
                observables=['agent1','agent2','agent3','agent4','agent5' ,'Glc_2', 'Starch'],
                actions=["Amylase_2_e"],
                gamma=1,
                tau=0.1
                )



agent3=tk.Agent("agent3",
                model=toy_model_3,
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=4,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
				observables=['agent1','agent2','agent3','agent4','agent5' ,'Glc_3', 'Starch'],
                actions=["Amylase_3_e"],
                gamma=1,
                tau=0.1
)

agent4=tk.Agent("agent4",
                model=toy_model_4,
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=4,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
				observables=['agent1','agent2','agent3','agent4','agent5' ,'Glc_4', 'Starch'],
                actions=["Amylase_4_e"],
                gamma=1,
                tau=0.1
)

agent5=tk.Agent("agent5",
                model=toy_model_5,
                actor_network=tk.NN,
                critic_network=tk.NN,
                clip=0.1,
                lr_actor=0.0001,
                lr_critic=0.001,
                grad_updates=4,
                optimizer_actor=torch.optim.Adam,
                optimizer_critic=torch.optim.Adam,
				observables=['agent1','agent2','agent3','agent4','agent5' ,'Glc_5', 'Starch'],
                actions=["Amylase_5_e"],
                gamma=1,
                tau=0.1
)

agents=[agent1,agent2,agent3,agent4,agent5]

env=tk.Environment(name="Toy-Exoenzyme-Five-agents-mass-transfer-low",
                    agents=agents,
                    dilution_rate=0.0001,
                    initial_condition={"Glc":0,"Glc_1":20,"Glc_2":20,"Glc_3":20,"Glc_4":20,"Glc_5":20,"agent1":0.1,"agent2":0.1,'agent3':0.1,'agent4':0.1,'agent5':0.1,"Starch":10},
                    inlet_conditions={"Starch":10},
                    extracellular_reactions=[{"reaction":{
                    "Glc_1":10,
                    "Starch":-0.1,},
                    "kinetics": (tk.general_kinetic,("Starch","Amylase_1"))} ### The trick
		            ,
		            {
                    "reaction":{
                    "Glc_2":10,
                    "Starch":-0.1,},
                    "kinetics": (tk.general_kinetic,("Starch","Amylase_2")) ### The trick
                    ,}
		    ,
		    {
                    "reaction":{
                    "Glc_3":10,
                    "Starch":-0.1,},
                    "kinetics": (tk.general_kinetic,("Starch","Amylase_3")) ### The trick
		    
            },
            {
                    "reaction":{
                    "Glc_4":10,
                    "Starch":-0.1,},
                    "kinetics": (tk.general_kinetic,("Starch","Amylase_4")) ### The trick
                    
            },  
	    
            {   "reaction":{
                    "Glc_5":10,
                    "Starch":-0.1,},
                    "kinetics": (tk.general_kinetic,("Starch","Amylase_5")) ### The trick
                    
            }  ,
            {   
                "reaction":{
                    "Glc_1":-1,
                    "Glc":1},
                "kinetics": (tk.mass_transfer,("Glc_1","Glc"))    ### The trick
            }
            ,
            {
                "reaction":{
                    "Glc_2":-1,
                    "Glc":1},
                "kinetics": (tk.mass_transfer,("Glc_2","Glc")) ### The trick
            }
            ,
            {
                "reaction":{
                    "Glc_3":-1,
                    "Glc":1},
                "kinetics": (tk.mass_transfer,("Glc_3","Glc"))  ### The trick
            },
            {
                "reaction":{
                    "Glc_4":-1,
                    "Glc":1},
                "kinetics": (tk.mass_transfer,("Glc_4","Glc"))  ### The trick
            }
            ,
            {
                "reaction":{ 
                    "Glc_5":-1,
                    "Glc":1},
                "kinetics": (tk.mass_transfer,("Glc_5","Glc"))   ### The trick
            }
					],
                           dt=0.1,
                           number_of_batches=5000,
                           episodes_per_batch=int(NUM_CORES/2),
                           )
